In [ ]:
import pandas as pd

In [ ]:
segdup = pd.read_csv("../data/segdup_regions.csv")[["chrom", "chromStart", "chromEnd", "fracMatch", "fracMatchIndel", "otherChrom", "otherStart", "otherEnd"]]

In [ ]:
segdup

## convert to bed

In [ ]:
segdup = segdup[(segdup.fracMatch > 0.98) & (~segdup.chrom.str.contains("_"))]

In [ ]:
segdup["chrom"] = segdup["chrom"].replace({"chrX": "chr23", "chrY": "chr24", "chrM": "chr25"})
segdup["chrom"] = segdup["chrom"].str.replace("chr", "").astype("int")
segdup = segdup.sort_values(by=["chrom", "chromStart", "chromEnd"])
segdup["chrom"] = "chr" + segdup["chrom"].astype(str)
segdup["chrom"] = segdup["chrom"].replace({"chr23": "chrX", "chr24": "chrY", "chr25": "chrM"})

In [ ]:
segdup

In [ ]:
segdup.otherChrom.value_counts()

In [ ]:
segdup[["chrom", "chromStart", "chromEnd"]].to_csv("../data/segDup_majorAllele_withAltContigs_98pcFracMatch.bed", sep="\t", header=False, index=False)

### a lot of segments in this bed file overlap with each other. had to run bedtools merge to get the union of those segments

## see how many genes in the CN matrix are going to be masked

In [ ]:
from taigapy import TaigaClient
tc = TaigaClient()

OmicsCNGene = tc.get(name='internal-23q2-1e49', version=97, file='OmicsCNGene')


In [ ]:
from mgenepy.utils import helper as h

mybiomart = h.generateGeneNames(attributes=["start_position", "end_position", "chromosome_name"],)
mybiomart = mybiomart.rename(
        columns={
            "start_position": "start",
            "end_position": "end",
            "chromosome_name": "Chromosome",
        }
    )

In [ ]:
mybiomart

In [ ]:
mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({"X": "23", "Y": "24", "MT": "25"})
mybiomart = mybiomart[mybiomart.Chromosome.isin(set(map(str, range(1, 26))))]
mybiomart["Chromosome"] = mybiomart["Chromosome"].astype(int)

In [ ]:
mybiomart = mybiomart.sort_values(by=["Chromosome", "start", "end"])
mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({23: "X", 24: "Y", 25: "MT"})
mybiomart = mybiomart.drop_duplicates("hgnc_symbol", keep="first")
mybiomart["gene_name"] = [
        i["hgnc_symbol"] + " (" + str(i["entrezgene_id"]).split(".")[0] + ")"
        for _, i in mybiomart.iterrows()
    ]
mybiomart["Chromosome"] = "chr" + mybiomart["Chromosome"].astype(str)


In [ ]:
mybiomart[mybiomart.gene_name.isin(OmicsCNGene)][["Chromosome", "start", "end", "gene_name"]].to_csv("../data/biomart_cngenes.bed", sep="\t", header=False, index=False)

In [ ]:
gene_dict = mybiomart[["Chromosome", "start", "end", "gene_name"]].set_index('gene_name').T.to_dict('list')

In [ ]:
gene_dict

In [ ]:
overlap_df = pd.read_csv("/home/xiaomeng/bin/depmap_omics/data/overlap_withalt.txt", sep="\t", names=["chrom", "start", "end", "gene_name"])

In [ ]:
overlap_df

In [ ]:
masked_genes = []
for g in overlap_df.gene_name.unique().tolist():
    _, start, end = gene_dict[g]
    gene_length = end - start
    overlap_length = 0
    overlap_segments = overlap_df[overlap_df.gene_name == g]
    for i, v in overlap_segments.iterrows():
        overlap_length += v['end'] - v['start']
    if overlap_length / gene_length > 0.5:
        masked_genes.append(g)

In [ ]:
masked_genes

In [ ]:
len(masked_genes)

In [ ]:
[c for c in OmicsCNGene.columns if c.startswith("LINC01670")]

In [ ]:
gene_dict["LINC01670 (105379487)"]

In [ ]:
OmicsCNGene[masked_genes].sum().value_counts()

In [ ]:
OmicsCNGene.sum().value_counts()

# a proper function for it

In [ ]:
import pybedtools

In [ ]:
def maskSegDup(cnmatrix, matname, mybiomart, save_output="", segdup_bed="../data/repeatMasker_max10_noAlt_merged.bed", thresh=0.5):
    # sort and format biomart
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({"X": "23", "Y": "24", "MT": "25"})
#     mybiomart = mybiomart[mybiomart.Chromosome.isin(set(map(str, range(1, 26))))]
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].astype(int)
#     mybiomart = mybiomart.sort_values(by=["Chromosome", "start", "end"])
#     mybiomart["Chromosome"] = mybiomart["Chromosome"].replace({23: "X", 24: "Y", 25: "MT"})
#     mybiomart = mybiomart.drop_duplicates("hgnc_symbol", keep="first")
#     mybiomart["Chromosome"] = "chr" + mybiomart["Chromosome"].astype(str)
    
    mybiomart[mybiomart.gene_name.isin(cnmatrix)][["Chromosome", "start", "end", "gene_name"]].to_csv(save_output + "biomart_cngenes.bed", sep="\t", header=False, index=False)
    
    cngenes = pybedtools.BedTool(save_output + "biomart_cngenes.bed")
    cngenes.intersect(segdup_bed).saveas(save_output + 'mask_overlap.bed')
    
    overlap_df = pd.read_csv(save_output + 'mask_overlap.bed', sep="\t", names=["chrom", "start", "end", "gene_name"])
    gene_dict = mybiomart[["Chromosome", "start", "end", "gene_name"]].set_index('gene_name').T.to_dict('list')
    masked_genes = []
    for g in overlap_df.gene_name.unique().tolist():
        _, start, end = gene_dict[g]
        gene_length = end - start
        overlap_length = 0
        overlap_segments = overlap_df[overlap_df.gene_name == g]
        for i, v in overlap_segments.iterrows():
            overlap_length += v['end'] - v['start']
        if overlap_length / gene_length > thresh:
            masked_genes.append(g)
    print("masking " + str(len(masked_genes)) + " genes from CN matrix")
    cnmatrix = cnmatrix.drop(columns=masked_genes)
    
    return cnmatrix, masked_genes

In [ ]:
genes = pybedtools.BedTool("../data/biomart_cngenes.bed")

In [ ]:
genes.intersect("/home/xiaomeng/bin/depmap_omics/data/segDup_majorAllele_98pcFracMatch_merged.bed").saveas('ov.bed')

In [ ]:
cnmatrix, masked_genes = maskSegDup(OmicsCNGene, "CN", mybiomart, thresh=0.01, segdup_bed="../data/repeatMasker_max10_noAlt_merged.bed")

In [ ]:
masked_genes

## repeatmasker

In [ ]:
repeats = pd.read_csv("/home/xiaomeng/bin/depmap_omics/data/repeatmasker.csv")[["milliDiv", "milliDel", "milliIns", "genoName", "genoStart", "genoEnd"]]

In [ ]:
repeats["max"] = repeats[["milliDiv", "milliDel", "milliIns"]].max(axis=1)

In [ ]:
repeats = repeats[(repeats["max"] < 10) & (~repeats.genoName.str.contains("_"))]
repeats["genoName"] = repeats["genoName"].replace({"chrX": "chr23", "chrY": "chr24", "chrM": "chr25"})
repeats["genoName"] = repeats["genoName"].str.replace("chr", "").astype("int")
repeats = repeats.sort_values(by=["genoName", "genoStart", "genoEnd"])
repeats["genoName"] = "chr" + repeats["genoName"].astype(str)
repeats["genoName"] = repeats["genoName"].replace({"chr23": "chrX", "chr24": "chrY", "chr25": "chrM"})

In [ ]:
repeats[["genoName", "genoStart", "genoEnd"]].to_csv("../data/repeatMasker_max10_noAlt.bed", sep="\t", header=False, index=False)

In [ ]:
repeats

In [ ]:
repeats[(repeats.genoName == "chr11") & (repeats.genoStart > 55635000) & (repeats.genoStart < 55641309)]